In [1]:
import os
%pwd
os.chdir("../")
%pwd


'd:\\Data Science\\END to END Proj\\BloodCellClassification'

In [2]:
%pip install dagshub
import dagshub
dagshub.init(repo_owner='gowtham-dd', repo_name='BloodCellClassification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Accessing as gowtham-dd

Initialized MLflow to track repo "gowtham-dd/BloodCellClassification"

Repository gowtham-dd/BloodCellClassification initialized!

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    mlflow_uri: str
    batch_size: int
    img_height: int
    img_width: int
    target_metric: str

In [4]:
from BloodCellClassifier.constant import *
from BloodCellClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])



    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_evaluation

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            metric_file_name=Path(config.metric_file_name),
            mlflow_uri=config.mlflow_uri,
            batch_size=params.batch_size,
            img_height=params.img_height,
            img_width=params.img_width,
            target_metric=params.target_metric
        )

In [9]:
import os
import mlflow
import mlflow.tensorflow
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from urllib.parse import urlparse
from pathlib import Path
from BloodCellClassifier.utils.common import save_json
from BloodCellClassifier import logger

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
        self.image_size = (self.config.img_height, self.config.img_width)

    def _load_test_generator(self):
        """Create test data generator from saved CSV"""
        try:
            test_df = pd.read_csv(self.config.test_data_path)
            
            datagen = ImageDataGenerator(
                preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
            )
            
            test_gen = datagen.flow_from_dataframe(
                dataframe=test_df,
                x_col="filepaths",
                y_col="labels",
                target_size=self.image_size,
                batch_size=self.config.batch_size,
                class_mode="categorical",
                shuffle=False
            )
            return test_gen
        except Exception as e:
            logger.error(f"Error creating test generator: {e}")
            raise

    def _load_model(self):
        """Load the saved TensorFlow model"""
        try:
            return tf.keras.models.load_model(self.config.model_path)
        except Exception as e:
            logger.error(f"Error loading model: {e}")
            raise

    def evaluate_model(self):
        """Evaluate model and return metrics"""
        try:
            model = self._load_model()
            test_gen = self._load_test_generator()

            # Evaluate model
            loss, accuracy = model.evaluate(test_gen)
            
            metrics = {
                "loss": float(loss),
                "accuracy": float(accuracy)
            }

            # Save metrics
            save_json(Path(self.config.metric_file_name), metrics)
            logger.info(f"Evaluation metrics: {metrics}")
            return model, metrics

        except Exception as e:
            logger.error(f"Error during evaluation: {e}")
            raise

    def log_into_mlflow(self):
        """Log evaluation results to MLflow"""
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        try:
            with mlflow.start_run():
                # Load model and evaluate
                model, metrics = self.evaluate_model()

                # Log parameters
                mlflow.log_params({
                    "batch_size": self.config.batch_size,
                    "img_height": self.config.img_height,
                    "img_width": self.config.img_width
                })

                # Log metrics
                mlflow.log_metrics(metrics)

                # Log model
                if tracking_url_type_store != "file":
                    mlflow.tensorflow.log_model(
                        model,
                        artifact_path="blood_cell_model",
                        registered_model_name="BloodCell_CNN_Model"
                    )
                else:
                    mlflow.log_artifacts(str(self.config.model_path), artifact_path="blood_cell_model")

                logger.info("Model evaluation and MLflow logging completed successfully")

        except Exception as e:
            logger.error(f"MLflow logging failed: {e}")
            if mlflow.active_run():
                mlflow.end_run(status="FAILED")
            raise

In [8]:

try:
            config = ConfigurationManager()
            eval_config = config.get_model_evaluation_config()
            evaluator = ModelEvaluation(config=eval_config)
            evaluator.log_into_mlflow()
except Exception as e:
            logger.exception(e)
            raise e

[2025-08-01 18:58:12,601: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-01 18:58:12,612: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-01 18:58:12,614: INFO: common: created directory at: artifacts]
[2025-08-01 18:58:12,616: INFO: common: created directory at: artifacts/model_evaluation]
Found 2988 validated image filenames belonging to 4 classes.
 71/374 [====>.........................] - ETA: 5:13 - loss: 0.1730 - accuracy: 0.9225